<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko
!pip install psycopg2

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=9471447390c8829a6a24aef6f99c07a6794ca03440b3d48636df54f9f93ea246
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


## Setup SQLite


In [ ]:
#@title Important Directories { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from pycoingecko import CoinGeckoAPI
import sqlite3 as sq
import hashlib

import logging
import sys
from tqdm.notebook import tqdm

In [ ]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

In [ ]:
#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('deFi_lama_total_TVL',),
 ('deFi_lama_tvl',),
 ('deFi_lama_general',),
 ('deFi_lama_id',),
 ('proprietary_Twitter_id',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('proprietary_master_id',),
 ('CoinGecko_ohlc',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_Twitter_Partnership_ohlc',)]

# CoinGecko API Statistics Report

In [ ]:
#General Information table

def currency_stats(url):
    """
    Generates a list of the currencies on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    df_currency = pd.DataFrame(data, columns = ['id'])
    """
    print('Stats: ')
    print(df_currency.info(verbose=True))
    print(df_currency.isnull().sum())
    print(data)

    print(df_currency.describe())
    """
    return df_currency

def main():

    url = "https://api.coingecko.com/api/v3/simple/supported_vs_currencies"
    currency_stats(url)
    
if __name__ == "__main__":
    main()



In [44]:

url = "https://api.coingecko.com/api/v3/coins/list"


frames = []
response = requests.get(url)
data = response.json()


token_list = pd.DataFrame(data)
token_list_clean = pd.DataFrame() 
token_list_clean['CoinGecko_id'] = token_list['id']
symbol = token_list['symbol'].str.upper()
name = token_list['name'] 

token_list_clean['prop_id'] = symbol + '.' + name.str.replace(" ", "")
token_list_clean['symbol'] = symbol
token_list_clean['name'] = name

token_list_clean['prop_timestamp'] = int( time.time_ns() / 1000 )

list_anonymous = token_list_clean['prop_id']

for i in list_anonymous:
    text = i
    hash_object = hashlib.md5(text.encode())
    md5_hash = hash_object.hexdigest()
    token_list_clean['anon_id'] = str('ANON') + str(md5_hash.upper())



token_list_clean = token_list_clean.dropna()
token_list_clean.reset_index(drop=True, inplace=True)
token_list_clean.head()


    

,CoinGecko_id,prop_id,symbol,name,prop_timestamp,anon_id
0,01coin,ZOC.01coin,ZOC,01coin,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
1,0-5x-long-algorand-token,ALGOHALF.0.5XLongAlgorandToken,ALGOHALF,0.5X Long Algorand Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
2,0-5x-long-altcoin-index-token,ALTHALF.0.5XLongAltcoinIndexToken,ALTHALF,0.5X Long Altcoin Index Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
3,0-5x-long-ascendex-token-token,ASDHALF.0.5XLongAscendExTokenToken,ASDHALF,0.5X Long AscendEx Token Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
4,0-5x-long-bitcoin-cash-token,BCHHALF.0.5XLongBitcoinCashToken,BCHHALF,0.5X Long Bitcoin Cash Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8


In [51]:
#@title Load Token ID to SQLite { display-mode: "code" }

proprietary_master_id = "proprietary_master_id"

token_list_clean.to_sql(proprietary_master_id, conn_SQLite, if_exists='append', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

df_master_id = pd.read_sql('select distinct * from {}'.format(proprietary_master_id), conn_SQLite)

In [53]:
df_master_id


,CoinGecko_id,prop_id,symbol,name,prop_timestamp,anon_id
0,01coin,ZOC.01coin,ZOC,01coin,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
1,0-5x-long-algorand-token,ALGOHALF.0.5XLongAlgorandToken,ALGOHALF,0.5X Long Algorand Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
2,0-5x-long-altcoin-index-token,ALTHALF.0.5XLongAltcoinIndexToken,ALTHALF,0.5X Long Altcoin Index Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
3,0-5x-long-ascendex-token-token,ASDHALF.0.5XLongAscendExTokenToken,ASDHALF,0.5X Long AscendEx Token Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
4,0-5x-long-bitcoin-cash-token,BCHHALF.0.5XLongBitcoinCashToken,BCHHALF,0.5X Long Bitcoin Cash Token,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
...,...,...,...,...,...,...
13686,zyro,ZYRO.Zyro,ZYRO,Zyro,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
13687,zyrri,ZYR.Zyrri,ZYR,Zyrri,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
13688,zytara-dollar,ZUSD.ZytaraDollar,ZUSD,Zytara Dollar,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8
13689,zyx,ZYX.ZYX,ZYX,ZYX,1650675672916966,ANON8E73CBF79B0030CB158A7E48275337D8


# CoinGecko API Extraction

In [54]:
#@title  Download Coins Data Reports { display-mode: "code" }


url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)
id = token_list.id
id


#loop data extraction to token list
try:

  start = 0
  ends = 1500

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}".format(i))
    output = response.text
    json_parsed = json.loads(output)
    print(json_parsed)

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    #results = [full_table]
    frames.append(full_table)

    #Slow down requests

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
single['prop_timestamp'] = int( time.time_ns() / 1000 )

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/"
single.to_csv(path + 'CoinGecko_general_data_coins_{}-{}.csv'.format(start,ends))



{'id': '01coin', 'symbol': 'zoc', 'name': '01coin', 'asset_platform_id': None, 'platforms': {'': ''}, 'block_time_in_minutes': 0, 'hashing_algorithm': 'NeoScrypt', 'categories': ['Masternodes'], 'public_notice': None, 'additional_notices': [], 'localization': {'en': '01coin', 'de': '01coin', 'es': '01coin', 'fr': '01coin', 'it': '01coin', 'pl': '01coin', 'ro': '01coin', 'hu': '01coin', 'nl': '01coin', 'pt': '01coin', 'sv': '01coin', 'vi': '01coin', 'tr': '01coin', 'ru': '01coin', 'ja': '01coin', 'zh': '01coin', 'zh-tw': '01coin', 'ko': '01coin', 'ar': '01coin', 'th': '01coin', 'id': '01coin', 'cs': '01coin', 'da': '01coin', 'el': '01coin', 'hi': '01coin', 'no': '01coin', 'sk': '01coin', 'uk': '01coin', 'he': '01coin', 'fi': '01coin', 'bg': '01coin', 'hr': '01coin', 'lt': '01coin', 'sl': '01coin'}, 'description': {'en': '', 'de': '', 'es': '', 'fr': '', 'it': '', 'pl': '', 'ro': '', 'hu': '', 'nl': '', 'pt': '', 'sv': '', 'vi': '', 'tr': '', 'ru': '', 'ja': '', 'zh': '', 'zh-tw': '', 'k

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


                              id    symbol                           name  \
0                         01coin       zoc                         01coin   
0       0-5x-long-algorand-token  algohalf       0.5X Long Algorand Token   
0  0-5x-long-altcoin-index-token   althalf  0.5X Long Altcoin Index Token   

  asset_platform_id  block_time_in_minutes hashing_algorithm     categories  \
0              None                      0         NeoScrypt  [Masternodes]   
0          ethereum                      0              None             []   
0          ethereum                      0              None             []   

  public_notice additional_notices country_origin  ...  \
0          None                 []                 ...   
0          None                 []                 ...   
0          None                 []                 ...   

  market_data.price_change_percentage_30d_in_currency.xau  \
0                                                NaN        
0                  

In [59]:
single.info()

,id,symbol,name,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,public_notice,additional_notices,country_origin,...,market_data.price_change_percentage_30d_in_currency.xau,market_data.price_change_percentage_30d_in_currency.xdr,market_data.price_change_percentage_30d_in_currency.xlm,market_data.price_change_percentage_30d_in_currency.xrp,market_data.price_change_percentage_30d_in_currency.yfi,market_data.price_change_percentage_30d_in_currency.zar,market_data.price_change_percentage_30d_in_currency.bits,market_data.price_change_percentage_30d_in_currency.link,market_data.price_change_percentage_30d_in_currency.sats,prop_timestamp
0,01coin,zoc,01coin,None,0,NeoScrypt,[Masternodes],None,[],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1650676175860623
0,0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token,ethereum,0,None,[],None,[],,...,-10.85299,-8.76946,0.80352,3.59989,5.40419,-8.72097,1.94458,9.54425,1.94458,1650676175860623
0,0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token,ethereum,0,None,[],None,[],,...,0.66560,1.94005,2.78747,10.30911,-0.02557,0.62803,0.51613,1.80200,0.51613,1650676175860623


In [60]:
#@title Load Token ID to SQLite { display-mode: "code" }
token_ID  = "CoinGecko_general_data_coins"

single.to_sql(token_ID, conn_SQLite, index=False,  if_exists='append') # writes to file

# Save (commit) the changes
conn_SQLite.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.

df_CoinGecko_general_data_coins = pd.read_sql('select * from {}'.format(token_ID), conn_SQLite)





OperationalError: ignored

# Aggregating Output

In [72]:
import os
import glob
import pandas as pd
import sys
from pathlib import Path




path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/"


Cleaned_data_file = 'Bulk_CoinGecko_coins.csv'

df = pd.DataFrame()

extension = 'csv'
#dataset_location = os.chdir(r'D:\investing_programs\datasets\CoinMetrics')
all_filenames = []
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output/*.{}'.format(extension))]

#combine all files in the list
for i in all_filenames:
    df = pd.read_csv(i)#.read(),engine='openpyxl')   
    print(df)

combined_csv = df.append([pd.read_csv(f) for f in all_filenames])


combined_csv.to_csv( path + Cleaned_data_file, index=False, encoding='utf-8-sig')


print(combined_csv.head())

sys.exit()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (170,1648,1680,1688,1707,1717,1724,1725,1730,1739,1743,1744,1746,1747,1749,1751,1752,1753,1754,1756,1757) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


     Unnamed: 0                                id    symbol  \
0             0                            01coin       zoc   
1             0          0-5x-long-algorand-token  algohalf   
2             0     0-5x-long-altcoin-index-token   althalf   
3             0    0-5x-long-ascendex-token-token   asdhalf   
4             0      0-5x-long-bitcoin-cash-token   bchhalf   
..          ...                               ...       ...   
995           0  aston-martin-cognizant-fan-token        am   
996           0             aston-villa-fan-token       avl   
997           0              astra-guild-ventures       agv   
998           0                       astral-farm    astral   
999           0                          astralis      STAR   

                                 name    asset_platform_id  \
0                              01coin                  NaN   
1            0.5X Long Algorand Token             ethereum   
2       0.5X Long Altcoin Index Token             ethereu

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (109,1690,1694,1715,1727,1729,1736,1740,1744,1747,1748,1753,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


     Unnamed: 0                id   symbol            name  \
0             0  astra-protocol-2    $astr  Astra Protocol   
1             0        astrobirdz      abz      AstroBirdz   
2             0       astrodonkey     dnky     AstroDonkey   
3             0         astroelon  elonone       AstroElon   
4             0        astrofarms      leo      AstroFarms   
..          ...               ...      ...             ...   
994           0    bone-shibaswap     bone  Bone ShibaSwap   
995           0          boneswap     bone        BoneSwap   
996           0        bone-token     bone      Bone Token   
997           0         bonezyard      bnz       BonezYard   
998           0             bonfi      bnf           BonFi   

       asset_platform_id  block_time_in_minutes hashing_algorithm  \
0                    NaN                      0               NaN   
1    binance-smart-chain                      0               NaN   
2    binance-smart-chain                      0 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1429,1703,1725,1727,1730,1733,1734,1735,1737,1738,1740,1741,1744,1745,1746,1749,1750,1751,1752,1753,1754,1756,1757,1758,1759,1760,1761,1762,1763) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


     Unnamed: 0               id     symbol             name  \
0             0          bonfire    bonfire          Bonfire   
1             0     bongweedcoin        bwc     BongWeedCoin   
2             0     bonorum-coin       bono          Bonorum   
3             0        bontecoin      bonte        Bontecoin   
4             0       bonus-cake  BonusCake       Bonus Cake   
..          ...              ...        ...              ...   
994           0             cryn       cryn             CRYN   
995           0    cryowar-token       cwar          Cryowar   
996           0  crypcade-shares       cade  CrypCade Shares   
997           0         cryptalk       cryp         Cryptalk   
998           0         cryptaur        cpt         Cryptaur   

       asset_platform_id  block_time_in_minutes hashing_algorithm  \
0    binance-smart-chain                      0               NaN   
1    binance-smart-chain                      0               NaN   
2                    NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (170,1648,1680,1688,1707,1717,1724,1725,1730,1739,1743,1744,1746,1747,1749,1751,1752,1753,1754,1756,1757) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (109,1690,1694,1715,1727,1729,1736,1740,1744,1747,1748,1753,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DtypeWarning: Columns (1429,1703,1725,1727,1730,1733,1734,1735,1737,1738,1740,1741,1744,1745,1746,1749,1750,1751,1752,1753,1754,1756,1757,1758,1759,1760,1761,1762,1763) have mixed types.Specify dtype option on import or set low_memory=False.


   Unnamed: 0            id     symbol          name    asset_platform_id  \
0           0       bonfire    bonfire       Bonfire  binance-smart-chain   
1           0  bongweedcoin        bwc  BongWeedCoin  binance-smart-chain   
2           0  bonorum-coin       bono       Bonorum                  NaN   
3           0     bontecoin      bonte     Bontecoin                  NaN   
4           0    bonus-cake  BonusCake    Bonus Cake  binance-smart-chain   

   block_time_in_minutes hashing_algorithm categories public_notice  \
0                      0               NaN         []           NaN   
1                      0               NaN         []           NaN   
2                      1             Quark         []           NaN   
3                      0               NaN         []           NaN   
4                      0               NaN         []           NaN   

                                  additional_notices  ...  \
0                                                

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [73]:
combined_csv.describe()

,Unnamed: 0,block_time_in_minutes,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,...,market_data.roi.percentage,platforms.algorand,platforms.polkadot,platforms.eos,platforms.komodo,platforms.ardor,platforms.qtum,platforms.kusama,platforms.nem,platforms.bitshares
count,3997.0,3997.000000,1622.000000,1622.000000,1044.000000,3996.000000,3997.000000,3997.000000,3997.000000,3997.000000,...,211.000000,6.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.0,158.206905,70.367349,29.632670,1865.773946,6831.013514,7.581845,5.172541,10.433403,5.835226,...,1747.182112,3.641027e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,7838.707198,35.378260,35.378258,1111.882945,3874.968111,9.291181,14.029559,9.793341,12.172494,...,12596.422986,2.382405e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,-100.000000,1.370206e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.0,0.000000,50.000000,0.000000,850.000000,3500.500000,1.699000,0.000000,5.788000,1.000000,...,-97.974707,1.772643e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.0,0.000000,83.330000,16.670000,1895.500000,6981.000000,3.311000,0.000000,7.916000,1.000000,...,-89.423677,2.979956e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.0,0.000000,100.000000,50.000000,2824.000000,10082.750000,11.455000,0.000000,9.867000,1.662000,...,31.270486,5.674678e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,0.0,483044.000000,100.000000,100.000000,3766.000000,13560.000000,80.443000,98.939000,71.367000,100.036000,...,124778.614638,6.572919e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3997 entries, 0 to 998
Columns: 1784 entries, Unnamed: 0 to platforms.boba
dtypes: float64(1455), int64(12), object(317)
memory usage: 54.4+ MB


In [78]:
combined_csv.tail()

,Unnamed: 0,id,symbol,name,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,public_notice,additional_notices,...,platforms.defi-kingdoms-blockchain,platforms.oasis,platforms.ronin,platforms.fuse,platforms.nem,platforms.bitshares,platforms.meter,platforms.theta,platforms.hoo,platforms.boba
994,0,cryn,cryn,CRYN,ethereum,0,NaN,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,0,cryowar-token,cwar,Cryowar,solana,0,NaN,"['Solana Ecosystem', 'Gaming']",NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,0,crypcade-shares,cade,CrypCade Shares,binance-smart-chain,0,NaN,['Binance Smart Chain Ecosystem'],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,0,cryptalk,cryp,Cryptalk,binance-smart-chain,0,NaN,['Binance Smart Chain Ecosystem'],NaN,"[""Kindly be aware of <a href='https://www.coin...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,0,cryptaur,cpt,Cryptaur,ethereum,0,NaN,[],NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
